follow the same order of operation of summary, but produce a pivot table like the one already existing with:

In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder
from pandarallel import pandarallel

# show all columns
pd.set_option('display.max_rows', None)

In [2]:
# pick a country and import the conflicts
adm = 'E'
file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
filepath = os.path.join('.', 'adm_conflicts', adm, file)
df = pd.read_csv(filepath, low_memory=False)
display(df.head())
print(df.columns)

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.long_nom,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,...,TPA1.2055.475-2055.725_R,TPA1.401.89525-401.90475_R,TPA1.2237.0-2238.0_E,TPA1.2065.575-2065.825_R,TPA1.401.9501-401.9699_E,TPA1.401.8901-401.9099_R,TPA1.401.95525-401.96475_E,TPA1.401.8901-401.9099_E,TPA1.401.89525-401.90475_E,TPA1.2202.4-2203.4_E
0,116545157,NaN,E,NaN,PAZ_R,NaN,9.1/IA,23.12.2021,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113540649,NaN,E,NaN,DEIMOS-2,NaN,9.1/IA,06.03.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,113540649,NaN,E,NaN,DEIMOS-2,NaN,9.1/IA,06.03.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,116545157,NaN,E,NaN,PAZ_R,NaN,9.1/IA,23.12.2021,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,113540649,NaN,E,NaN,DEIMOS-2,NaN,9.1/IA,06.03.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.long_nom',
       ' com_el.prov', ' com_el.d_rcv', ' com_el.st_cur', ' orbit.orb_id',
       ' orbit.nbr_sat_pl', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' channel.bandwidth', ' channel.freq_min',
       ' channel.freq_max', 'tpaconflicts', 'percentoverlap',
       'TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R',
       'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R',
       'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R',
       'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R',
       'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E',

In [3]:
# function to extract the table
# Initialize pandarallel
num_logical_processors = os.cpu_count()
pandarallel.initialize(nb_workers=num_logical_processors, progress_bar=True)


def pivot_table(df):
    # Columns to keep
    cols = [
        ' com_el.sat_name',
        ' s_beam.beam_name',
        ' carrier_fr.freq_carr',
        ' channel.bandwidth',
        'tpaconflicts',
        'percentoverlap'
    ]
    lookupnames = [
        'UHFUP fc=401.96MHz BW=9.5kHz',
        'UHFUP fc=401.96MHz BW=19.8kHz',
        'SUP fc=2055.6MHz BW=250kHz',
        'UHFUP fc=401.90MHz BW=9.5kHz',
        'SDN fc=2237.5MHz BW=1MHz',
        'SUP fc=2065.7MHz BW=250kHz',
        'UHFDN fc=401.96MHz BW=19.8kHz',
        'UHFUP fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.96MHz BW=9.5kHz',
        'UHFDN fc=401.90MHz BW=19.8kHz',
        'UHFDN fc=401.90MHz BW=9.5kHz',
        'SDN fc=2202.9MHz BW=1MHz'
    ]

    df1 = df[cols].copy()
    if df1.empty:
        print("❗ DataFrame is empty")
        return pd.DataFrame()

    def functiontoapply(row):
        import pandas as pd
        satname = row[' com_el.sat_name'].strip()
        beamname = row[' s_beam.beam_name'].strip()
        full_id = f"{satname} - {beamname}"

        con = str(row['tpaconflicts']).split(':')
        percent_str = str(row['percentoverlap']).split(':')

        conflict_idxs = []
        for c in con[:-1]:
            conflict_idxs.append(int(c))
        percents = str(row['percentoverlap']).split(':')

        if not conflict_idxs or not percents:
            return pd.DataFrame(columns=['sat_beam', 'TPA1_beam', 'percent', 'count'])
        # else:
        #     print(conflict_idxs, percents)

        min_len = min(len(conflict_idxs), len(percents))
        fc = float(row[' carrier_fr.freq_carr'])
        bw = float(row[' channel.bandwidth'])  # Hz
        data = [
            {
                'sat_beam': full_id,
                'TPA1_beam': lookupnames[conflict_idxs[i]],
                'percent': percents[i],
                'count': 1,
                'fc': fc,
                'bw': bw
            }
            for i in range(min_len)
            if 0 <= conflict_idxs[i] < len(lookupnames)
        ]

        return pd.DataFrame(data)

    # Apply in parallel or serial depending on size
    if len(df1) > 24:
        expanded_rows = df1.parallel_apply(functiontoapply, axis=1)
    else:
        tqdm.pandas()
        expanded_rows = df1.apply(functiontoapply, axis=1)
    # tqdm.pandas()
    # expanded_rows = df1.apply(functiontoapply, axis=1)

    # Flatten the list of DataFrames
    df_expanded = pd.concat(expanded_rows.tolist(), ignore_index=True)

    if df_expanded.empty:
        print("⚠️ No valid conflict data found.")
        return pd.DataFrame()

    # Get the row with max percent per group
    df_expanded.sort_values('percent', ascending=False, inplace=True)
    grouped = df_expanded.groupby(['sat_beam', 'TPA1_beam'], as_index=False).first()

    # Combine into a readable string
    grouped['pivot_value'] = grouped.apply(
        lambda row: str(
            f"{row.percent}% @ fc={row.fc:.3f}MHz, BW={row.bw / 1e6:.3f}MHz" if row.bw >= 1e6 else f"{row.percent}% @ fc={row.fc:.3f}MHz, BW={row.bw / 1e3:.3f}kHz"),
        axis=1
    )

    # Create pivot
    pivot = grouped.pivot_table(
        index='sat_beam',
        columns='TPA1_beam',
        values='pivot_value',
        aggfunc='first',
        fill_value=''
    )

    return pivot


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
# usage

pivot_df = pivot_table(df)
pivot_df.to_csv('conflict_percent_pivot.csv')
display(pivot_df)


TPA1_beam,SDN fc=2237.5MHz BW=1MHz,SUP fc=2055.6MHz BW=250kHz,SUP fc=2065.7MHz BW=250kHz
sat_beam,,,
HISPASAT-LEO-ATL-A - 2SRX,,,"6.0% @ fc=2067.000MHz, BW=4.000MHz"
HISPASAT-LEO-ATL-A - 2STX,"10.0% @ fc=2240.000MHz, BW=5.000MHz",,
HISPASAT-LEO-ATL-A - SRX,,,"6.0% @ fc=2067.000MHz, BW=4.000MHz"
HISPASAT-LEO-ATL-A - STX,"10.0% @ fc=2240.000MHz, BW=5.000MHz",,
SATLNET-03 - SDL3,"11.0% @ fc=2235.000MHz, BW=5.100MHz",,
SATLNET-03 - SUL4,,"12.0% @ fc=2055.000MHz, BW=2.100MHz",


In [5]:
import os

# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'

# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# # todo comment this
# countries = ['ARG']

# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")

    # outfolder (must already exist)
    country_outfolder = os.path.join(outfolder, ccode)

    # Read data for the 'expanded_combined_tables_conflicts_lettersatnames.csv' file
    adm = ccode
    file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table
    summary_pivot = pivot_table(df)
    outpath = os.path.join(country_outfolder, 'summary_pivot.csv')
    summary_pivot.to_csv(outpath, index=True)
    print('Summary pivot saved to ', outpath)

    # Read data for the 'expanded_combined_tables_conflicts_othersatnames.csv' file
    file = 'expanded_combined_tables_conflicts_othersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)

    # Generate summary pivot table for other satellite names
    summary_pivot_othersatnames = pivot_table(df)
    outpath = os.path.join(country_outfolder, 'summary_pivot_othersatnames.csv')
    summary_pivot_othersatnames.to_csv(outpath, index=True)

    print('Summary pivot for other satellite names saved to ', outpath)



=== Processing ARG ===
❗ DataFrame is empty
Summary pivot saved to  adm_conflicts\ARG\summary_pivot.csv


Summary pivot for other satellite names saved to  adm_conflicts\ARG\summary_pivot_othersatnames.csv
